# AOC 2020: Day 8
This notebook contains the answers to advent of code 2020 day 8.

Import packages:

In [2]:
import pandas as pd

Load input:

In [86]:
boot_code = pd.read_csv('input_day8.txt', sep=" ", header=None)
boot_code.columns = ["instruction", "step"]

In [15]:
boot_code

,instruction,step
0,acc,37
1,acc,-4
2,nop,405
3,jmp,276
4,acc,39
...,...,...
618,acc,-11
619,acc,45
620,acc,0
621,acc,28


### Part 1
Run boot-code and count accumulator:

In [8]:
len(boot_code)

623

In [21]:
boot_code['instruction'][622]

'jmp'

In [28]:
def boot(boot_code):
    accumulator = 0
    index_counter = set()
    i = 0
    while i in range(len(boot_code)):
        inst = boot_code['instruction'][i]
        step = boot_code['step'][i]
        # check if index is looping
        if i in index_counter:
            break
        else:
            index_counter.add(i)
            if inst == 'acc':
                accumulator += step
                i += 1
            elif inst == 'jmp':
                i += step
            elif inst == 'nop':
                i += 1
    return accumulator, index_counter

In [29]:
accumulator, index_counter = boot(boot_code)

In [30]:
accumulator

1801

### Part 2
Find corrupted entry:

In [68]:
# BRUTE FORCE = BIG SAD

In [99]:
jump_index = boot_code['instruction']=='jmp'
nop_index = boot_code['instruction']=='nop'
jmp_nop_index = jump_index | nop_index

In [100]:
def boot_exit(boot_code, jmp_nop_index):
    accumulator = 0
    index_counter = set()
    i = 0
    switch = False
    while i in range(len(boot_code)):
        # check if index is looping, if so reset
        if i in index_counter:
            accumulator = 0
            index_counter = set()
            i = 0
            switch = False
        else:
            inst = boot_code['instruction'][i]
            step = boot_code['step'][i]
            # switch if not yet happened
            if jmp_nop_index[i] and not switch:
                if inst == 'jmp':
                    inst = 'nop'
                    jmp_nop_index[i] = False
                    switch = True
                elif inst == 'nop':
                    inst = 'jmp'
                    jmp_nop_index[i] = False
                    switch = True
            index_counter.add(i)
            if inst == 'acc':
                accumulator += step
                i += 1
            elif inst == 'jmp':
                i += step
            elif inst == 'nop':
                i += 1
    return accumulator, index_counter

In [101]:
acc, ind = boot_exit(boot_code, jmp_nop_index)

In [102]:
acc

2060

In [ ]:
# SMART FORCE = MORE CODE ?!

In [43]:
# find possible starting points: find instruction that exceeds bottom limit
index = set()
for i in range(len(boot_code)-1, -1, -1):
        inst = boot_code['instruction'][i]
        step = boot_code['step'][i]
        if inst == 'jmp' and i+step>622:
            index.add(i)

In [45]:
index # single entry/exit point

{622}

In [53]:
# find indexes to which a jump is possible
def find_jump_index(boot_code):
    jump_index = set()
    for i in range(len(boot_code)):
        inst = boot_code['instruction'][i]
        step = boot_code['step'][i]
        if inst == 'jmp':
            jump_index.add(i+step)
    return jump_index

In [54]:
jump_index = find_jump_index(boot_code)

In [55]:
jump_index

{8,
 13,
 20,
 25,
 32,
 41,
 47,
 53,
 54,
 56,
 63,
 71,
 78,
 80,
 81,
 84,
 87,
 92,
 93,
 99,
 107,
 111,
 115,
 122,
 127,
 135,
 142,
 147,
 154,
 157,
 163,
 164,
 173,
 179,
 188,
 196,
 202,
 206,
 208,
 210,
 212,
 215,
 221,
 228,
 232,
 238,
 242,
 251,
 254,
 261,
 263,
 269,
 274,
 279,
 285,
 293,
 294,
 302,
 308,
 309,
 312,
 316,
 321,
 326,
 330,
 334,
 340,
 348,
 354,
 357,
 363,
 372,
 376,
 379,
 381,
 386,
 389,
 391,
 395,
 400,
 410,
 412,
 417,
 420,
 422,
 427,
 435,
 437,
 443,
 445,
 449,
 452,
 456,
 467,
 476,
 480,
 484,
 486,
 494,
 498,
 501,
 507,
 512,
 515,
 521,
 528,
 534,
 536,
 538,
 542,
 547,
 556,
 563,
 565,
 568,
 575,
 579,
 582,
 585,
 591,
 596,
 597,
 600,
 606,
 612,
 618,
 623}

In [47]:
# backwards engineer PART I, find overlapping index, try part I with changed index
def reverse_boot(boot_code):
    accumulator = 0
    index_counter = set()
    i = len(boot_code) - 1
    while i in range(len(boot_code)):
        inst = boot_code['instruction'][i]
        step = boot_code['step'][i]
        # check if index is looping
        if i in index_counter:
            break
        else:
            index_counter.add(i)
            if inst == 'acc':
                accumulator += step
                i -= 1
            elif inst == 'jmp':
                i -= step
            elif inst == 'nop':
                i -= 1
    return accumulator, index_counter